In [1]:
import os

# ---------- INITIALIZATION ----------
LC = 0  # Location Counter

mnemonics = {
    'STOP': ('00', 'IS', 0),
    'ADD': ('01', 'IS', 2),
    'SUB': ('02', 'IS', 2),
    'MULT': ('03', 'IS', 2),
    'MOVER': ('04', 'IS', 2),
    'MOVEM': ('05', 'IS', 2),
    'COMP': ('06', 'IS', 2),
    'BC': ('07', 'IS', 2),
    'DIV': ('08', 'IS', 2),
    'READ': ('09', 'IS', 1),
    'PRINT': ('10', 'IS', 1)
}

REG = {'AREG': 1, 'BREG': 2, 'CREG': 3, 'DREG': 4}

# ---------- FILE SETUP ----------
input_path = "text.txt"
file = open(input_path)

ifp = open("inter_code.txt", "w")  # intermediate code
lit = open("literals.txt", "w+")
tmp = open("tmp.txt", "w+")
sym = open("SymTab.txt", "w+")
poolf = open("PoolTab.txt", "w+")

symtab = {}
pooltab = []
words = []
symindex = 0


# ---------- TABLE PRINTERS ----------
def littab():
    print("\nLiteral Table:")
    lit.seek(0)
    for x in lit:
        print(x.strip())


def pooltab2():
    print("Pool Table:", pooltab)


def symbol():
    print("Symbol Table:", symtab)


# ---------- DIRECTIVE HANDLERS ----------
def END():
    global LC
    pool = 0
    z = 0
    ifp.write("\t(AD,02)\n")

    lit.seek(0)
    for x in lit:
        if "**" in x:
            pool += 1
            if pool == 1:
                pooltab.append(z)
            y = x.split()
            tmp.write(y[0] + "\t" + str(LC) + "\n")
            LC += 1
        else:
            tmp.write(x)
        z += 1

    lit.truncate(0)
    tmp.seek(0)
    for x in tmp:
        lit.write(x)
    tmp.truncate(0)


def LTORG():
    global LC
    pool = 0
    z = 0
    lit.seek(0)
    x = lit.readlines()
    i = 0
    while i < len(x):
        f = []
        if "**" in x[i]:
            j = 0
            pool += 1
            if pool == 1:
                pooltab.append(z)
            while x[i][j] != "'":
                j += 1
            j += 1
            while x[i][j] != "'":
                f.append(x[i][j])
                j += 1
            ifp.write(f"\t(AD,05)\t(DL,02)(C,{''.join(f)})\n")
            y = x[i].split()
            tmp.write(y[0] + "\t" + str(LC) + "\n")
            LC += 1
        else:
            tmp.write(x[i])
            z += 1
        i += 1

    lit.truncate(0)
    tmp.seek(0)
    for x in tmp:
        lit.write(x)
    tmp.truncate(0)


def ORIGIN(addr):
    global LC
    ifp.write(f"\t(AD,03)\t(C,{addr})\n")
    LC = int(addr)


def DS(size):
    global LC
    ifp.write(f"\t(DL,01)\t(C,{size})\n")
    LC += int(size)


def DC(value):
    global LC
    ifp.write(f"\t(DL,02)\t(C,{value})\n")
    LC += 1


# ---------- MNEMONIC HANDLER ----------
def OTHERS(mnemonic, k):
    global words, mnemonics, symtab, LC, symindex

    if mnemonic not in mnemonics:
        print(f"⚠️ Unknown mnemonic '{mnemonic}' at LC={LC}, skipping line.")
        return

    z = mnemonics[mnemonic]
    ifp.write(f"\t({z[1]},{z[0]})\t")
    y = z[-1]

    for i in range(1, y + 1):
        words[k + i] = words[k + i].replace(",", "")
        if words[k + i] in REG.keys():
            ifp.write(f"(RG,{REG[words[k + i]]})")
        elif "=" in words[k + i]:
            lit.seek(0, 2)
            lit.write(words[k + i] + "\t**\n")
            lit.seek(0)
            x = lit.readlines()
            ifp.write(f"(L,{len(x)})")
        else:
            if words[k + i] not in symtab.keys():
                symtab[words[k + i]] = ("**", symindex)
                ifp.write(f"(S,{symindex})")
                symindex += 1
            else:
                w = symtab[words[k + i]]
                ifp.write(f"(S,{w[-1]})")
    ifp.write("\n")
    LC += 1


def detect_mn(k):
    global words, LC
    if words[k] == "START":
        LC = int(words[1])
        ifp.write(f"\t(AD,01)\t(C,{LC})\n")
    elif words[k] == "END":
        END()
    elif words[k] == "LTORG":
        LTORG()
    elif words[k] == "ORIGIN":
        ORIGIN(words[k + 1])
    elif words[k] == "DS":
        DS(words[k + 1])
    elif words[k] == "DC":
        DC(words[k + 1])
    else:
        OTHERS(words[k], k)

    littab()
    pooltab2()
    symbol()


# ---------- MAIN LOOP ----------
for line in file:
    if not line.strip():
        continue

    words = line.split()
    if LC > 0:
        ifp.write(str(LC))
        print(f"\nLC = {LC}  →  {line.strip()}")

    k = 0
    if words[0] in mnemonics.keys():
        print("Mnemonic:", words[0])
        detect_mn(k)
    else:
        print("Label:", words[0], "Mnemonic:", words[1])
        if words[k] not in symtab.keys():
            symtab[words[k]] = (LC, len(symtab))
        else:
            x = symtab[words[k]]
            if x[0] == "**":
                symtab[words[k]] = (LC, x[1])
        k = 1
        detect_mn(k)

# ---------- CLEANUP ----------
ifp.close()
lit.seek(0)
sym.truncate(0)
for x in symtab:
    sym.write(f"{x}\t{symtab[x][0]}\n")
sym.close()

poolf.truncate(0)
for x in pooltab:
    poolf.write(f"{x}\n")
poolf.close()

lit.close()
tmp.close()
file.close()
if os.path.exists("tmp.txt"):
    os.remove("tmp.txt")

print("\n✅ Pass-1 completed successfully. Check inter_code.txt, SymTab.txt, PoolTab.txt, and literals.txt.")


Label: START Mnemonic: 100
⚠️ Unknown mnemonic '100' at LC=0, skipping line.

Literal Table:
Pool Table: []
Symbol Table: {'START': (0, 0)}
Label: A Mnemonic: DC

Literal Table:
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1)}

LC = 1  →  MOVER AREG, B
Mnemonic: MOVER

Literal Table:
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1), 'B': ('**', 0)}

LC = 2  →  MOVEM BREG, ='1'
Mnemonic: MOVEM

Literal Table:
='1'	**
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1), 'B': ('**', 0)}

LC = 3  →  ADD AREG, ='2'
Mnemonic: ADD

Literal Table:
='1'	**
='2'	**
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1), 'B': ('**', 0)}

LC = 4  →  SUB BREG, ='1'
Mnemonic: SUB

Literal Table:
='1'	**
='2'	**
='1'	**
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1), 'B': ('**', 0)}

LC = 5  →  B  DC 20
Label: B Mnemonic: DC

Literal Table:
='1'	**
='2'	**
='1'	**
Pool Table: []
Symbol Table: {'START': (0, 0), 'A': (0, 1), 'B': (5, 0)}

LC = 6  →  ORIGIN 30

IndexError: list index out of range